# Homework 1

**Homework:**

1.   Format the dataset the right way and encode it with the `TransactionEncoder`. 
2.  Run `apriori` or `fp-growth` algorithm on the data
3.   List `association rules` with reasonable `thresholds` and add a column with the `interest scores`
4.   Mine some interesting/relevant `association rules` (2 or 3 is enough, but you can do more if you want), which does not include bananas. Give a short reasoning, why it is important/relevant based on the metrics.

**Deadline: 2022.04.22. 23:59**

## Libraries

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

In [ ]:
#!pip install mlxtend -U

     |████████████████████████████████| 1.3 MB 13.2 MB/s 
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/

## Importing data

In [ ]:
orders = pd.read_csv('./order_products__train.csv')
products = pd.read_csv('./products.csv', index_col='product_id')

In [ ]:
orders

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
...,...,...,...,...
1384612,3421063,14233,3,1
1384613,3421063,35548,4,1
1384614,3421070,35951,1,1
1384615,3421070,16953,2,1


In [ ]:
products

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13
...,...,...,...
49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49685,En Croute Roast Hazelnut Cranberry,42,1
49686,Artisan Baguette,112,3


In [ ]:
# joining product names for convenience
orders = pd.merge(orders, products, how='left', on='product_id')

In [ ]:
# grouping by baskets
groups = orders.groupby('order_id')['product_name'].groups

In [ ]:
# making list of lists (baskets)
baskets = []
for group in groups.values():
    basket = list(orders.product_name[group].values)
    baskets.append(basket)

In [ ]:
baskets[:3]

[['Bulgarian Yogurt',
  'Organic 4% Milk Fat Whole Milk Cottage Cheese',
  'Organic Celery Hearts',
  'Cucumber Kirby',
  'Lightly Smoked Sardines in Olive Oil',
  'Bag of Organic Bananas',
  'Organic Hass Avocado',
  'Organic Whole String Cheese'],
 ['Grated Pecorino Romano Cheese',
  'Spring Water',
  'Organic Half & Half',
  'Super Greens Salad',
  'Cage Free Extra Large Grade AA Eggs',
  'Prosciutto, Americano',
  'Organic Garnet Sweet Potato (Yam)',
  'Asparagus'],
 ['Shelled Pistachios',
  'Organic Biologique Limes',
  'Organic Raw Unfiltered Apple Cider Vinegar',
  'Organic Baby Arugula',
  'Organic Hot House Tomato',
  'Green Peas',
  'Bunched Cilantro',
  'Flat Parsley, Bunch',
  'Fresh Dill']]

# Encoding

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(baskets).transform(baskets)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,#2 Coffee Filters,#2 Cone White Coffee Filters,#2 Mechanical Pencils,#4 Natural Brown Coffee Filters,& Go! Hazelnut Spread + Pretzel Sticks,+Energy Black Cherry Vegetable & Fruit Juice,0 Calorie Acai Raspberry Water Beverage,0 Calorie Fuji Apple Pear Water Beverage,0 Calorie Strawberry Dragonfruit Water Beverage,0% Fat Black Cherry Greek Yogurt y,...,with Sweet Cinnamon Bunches Cereal,with Xylitol Cinnamon 18 Sticks Sugar Free Gum,with Xylitol Island Berry Lime 18 Sticks Sugar Free Gum,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum,with Xylitol Original Flavor 18 Sticks Sugar Free Gum,with Xylitol Unwrapped Original Flavor 50 Sticks Sugar Free Gum,with Xylitol Unwrapped Spearmint 50 Sticks Sugar Free Gum,with Xylitol Watermelon Twist 18 Sticks Sugar Free Gum,with a Splash of Mango Coconut Water,with a Splash of Pineapple Coconut Water
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
131207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Association rule mining

In [ ]:
# using apriori algorithm
freq_its = apriori(df, min_support=0.004, use_colnames=True, low_memory=True)
freq_its

,support,itemsets
0,0.004207,(100 Calorie Per Bag Popcorn)
1,0.009893,(100% Raw Coconut Water)
2,0.009016,(100% Recycled Paper Towels)
3,0.017514,(100% Whole Wheat Bread)
4,0.004382,(2% Reduced Fat DHA Omega-3 Reduced Fat Milk)
...,...,...
528,0.004138,"(Seedless Red Grapes, Strawberries)"
529,0.004474,"(Organic Strawberries, Bag of Organic Bananas,..."
530,0.004047,"(Bag of Organic Bananas, Organic Raspberries, ..."
531,0.005411,"(Bag of Organic Bananas, Organic Strawberries,..."


In [ ]:
# association rule
ar = association_rules(freq_its, metric="confidence", min_threshold=0.05)
# interest score implementation
ar['interest_score'] = (ar['confidence'] - ar['consequent support']).abs()
# applying sorting based on interest_score
ar.sort_values(by='interest_score', ascending=False, inplace=True)
# changing frozenset datatype to single string
for col in ['antecedents', 'consequents']:
    ar[col] = ar[col].apply(lambda x: ', '.join(list(x))).astype("unicode").str.lower()
# displaying dataframe
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest_score
314,"organic hass avocado, organic raspberries",bag of organic bananas,0.007766,0.117980,0.004047,0.521099,4.416854,0.003131,1.841760,0.403119
319,"organic strawberries, organic hass avocado",bag of organic bananas,0.011729,0.117980,0.005411,0.461339,3.910321,0.004027,1.637430,0.343359
323,"organic raspberries, bag of organic bananas",organic strawberries,0.013566,0.083028,0.004946,0.364607,4.391379,0.003820,1.443157,0.281579
324,"organic strawberries, organic raspberries",bag of organic bananas,0.012728,0.117980,0.004946,0.388623,3.293980,0.003445,1.442678,0.270643
47,organic navel orange,bag of organic bananas,0.015090,0.117980,0.005526,0.366162,3.103598,0.003745,1.391554,0.248182
...,...,...,...,...,...,...,...,...,...,...
13,large lemon,bag of organic bananas,0.062000,0.117980,0.008056,0.129932,1.101311,0.000741,1.013738,0.011953
14,bag of organic bananas,large lemon,0.117980,0.062000,0.008056,0.068282,1.101311,0.000741,1.006742,0.006281
101,banana,organic hass avocado,0.142719,0.055583,0.007217,0.050571,0.909834,-0.000715,0.994721,0.005012
70,strawberries,bag of organic bananas,0.049494,0.117980,0.005922,0.119649,1.014148,0.000083,1.001896,0.001669


Many of the observations contain some kind of banana.

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/

In [ ]:
# filtering rule to exclude any set that contains the word banana
condition = ~(ar.antecedents.str.contains('banana') | ar.consequents.str.contains('banana'))
result = ar[condition]
result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest_score
128,bunched cilantro,limes,0.014633,0.045980,0.004253,0.290625,6.320672,0.003580,1.344874,0.244645
136,jalapeno peppers,limes,0.014473,0.045980,0.004169,0.288046,6.264590,0.003503,1.340003,0.242066
169,organic cilantro,limes,0.026873,0.045980,0.007675,0.285593,6.211228,0.006439,1.335401,0.239613
288,organic raspberries,organic strawberries,0.042268,0.083028,0.012728,0.301118,3.626710,0.009218,1.312056,0.218090
139,limes,large lemon,0.045980,0.062000,0.012156,0.264379,4.264159,0.009305,1.275113,0.202379
...,...,...,...,...,...,...,...,...,...,...
300,organic strawberries,raspberries,0.083028,0.024991,0.004291,0.051680,2.067965,0.002216,1.028144,0.026689
291,organic strawberries,organic red onion,0.083028,0.029099,0.004550,0.054801,1.883279,0.002134,1.027192,0.025702
236,strawberries,organic baby spinach,0.049494,0.074568,0.004954,0.100092,1.342296,0.001263,1.028363,0.025524
134,organic strawberries,honeycrisp apple,0.083028,0.027064,0.004359,0.052506,1.940089,0.002112,1.026852,0.025442


In [ ]:
result.query('lift > 6 and leverage > 0.005')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,interest_score
169,organic cilantro,limes,0.026873,0.045980,0.007675,0.285593,6.211228,0.006439,1.335401,0.239613
268,organic garlic,organic yellow onion,0.031690,0.032696,0.006478,0.204425,6.252314,0.005442,1.215856,0.171729
267,organic yellow onion,organic garlic,0.032696,0.031690,0.006478,0.198135,6.252314,0.005442,1.207573,0.166445
168,limes,organic cilantro,0.045980,0.026873,0.007675,0.166915,6.211228,0.006439,1.168101,0.140042


Above I used the `lift` and `leverage` metrics to find association rules as they provide strong information. If we look on the findings we can see limes and cilantro are commonly bougth together (vice versa) which makes sense as they are both basic ingredients of Mexican cousine. The other finding is the garlic and yellow onion pair (vice versa) which are basic ingredients in every cousine.